# Purely Functional Data Structures

### Terminology

#### Evaluation
**call-by-value** is a *strict evaluation* scheme which for a call `f x` evaluates an expression `x` before the call to a function `f`

**call-by-name** is a *lazy evaluation* scheme which for a call `f x` does not immediately evaluate an expression `x` but rather capture its environment and evaluates it when it's needed by the code of function `f`. However, each time `f` is called, so is `x` evaluated.

This scheme can be used to yield *amortized bounds* for *ephemeral* data structures, howeven, for *persistent* ones one could call certain operation multiple times and the amortized bound may collapse to the worst case bound if the operation is expensive.

**call-by-need** is a *lazy evaluation and memoization* scheme which behaves similarly to *call-by-name* with the important difference that once `x` is evaluated, it is also memoized and thus successive calls to `f` do not re-evaluate `x`.

This scheme is required to retain *amortized bounds* for *persistent* data structures for which one assumes arbitrary logical futures of operations (operation sequencing).

#### Persistence
 - **Persistent** data structure - *always* preserves the previous version of itself when it is modified (even concurrently)
 - **Ephemeral** data structure - might not preserve its complexity characteristics when accessed and modified concurrently even when implemented as immutable (especially amortized times)

#### Rebuilding
**Batched rebuilding** is a balancing technique which restores a *perfect balance* for a data structure after a sequnce of updates rather than after each operation. This approach can yield the same amortized bounds as the base one given
 1. rebuilding is not too frequent
 1. individual updates do not significantly degrade the performance of successive ones

*See formal definition in the book.*

**Global rebuilding** is a technique extending *batched rebuilding* which eliminates amortized bounds by executing the rebuilding transformation incrementally (few steps per operation).

We maintain two copies of the data structure:
 1. all queris and updates operate on a *primary / working copy*
 2. while rebuilding is done on a *secondary copy*.

When the rebuilding is finished these copies are swapped and updates to the previous working copy that have been buffered over time since the last swap are applied to the new one (to make it up to date).

Next rebuilding might start immediately of after a while.

**Lazy rebuilding** is a variant of *global rebuilding* which does not immediately "execute" the rebuilding transformation concurrently with normal operation but rather "pays for" the rebuilding concurrenly. The "execution" is postponed to some later time when it's been "paid for".

Furthermore, with nested suspensions (created by lazy operations) it is often possible to maintain single lazy data structure in which the evaluated and memoized portion (the one that's been "paid for") represents the *working* part and the rest is an analogy to the *secondary copy*.

Similarly to *global rebuilding* this approach is suitable for *persistent* data structures. On the other hand, it typically gives only amortized bounds (similarly to *batched rebuilding*). Fortunately, worst case bounds can usually be recovered by combining this with *scheduling* - i.e. *lazy rebuilding* with *scheduling* is an instance of *global rebuilding*.

#### Scheduling
**Scheduling** is a technique in which a data structure maintains a collection of unevaluated suspensions (results of lazy operations) - a *schedule* - for execution at specific (controlled) time.

Some members of the schedule may point to suspensions that have already been evaluated (in a different logical future - operation sequencing) but due to memization this can only result in a speed-up.

This technique is typically used in combination with *lazy rebuilding* to make amortized bounds worst case for persistent data structures.

#### Real-time data structure
By a **real-time** data structure one can understand a typically *persistent* data structure with efficient *worst case bounds* on its operations (or at least those of interest).

Typical areas where these data structures are used are:
 - *real-time systems* - if an operation misses a hard deadline and causes a system failure then it doesn't matter how many operations finished ahead of a schedule
 - *parallel systems* - if a processor in a synchonous system executes an expensive operation than all the other processors must wait until it completes
 - *interactive systems* - users often value consistency more than raw speed (single extremely slow operation in a sequence is more noticeable than if all operations the sequence are just slightly slower than normal even though the latter sequence may take longer in total)

## Queues

|       instance      | persistence | amortization | empty | isEmpty |     snoc     | head |     tail     |
|:-------------------:|:-----------:|:------------:|:-----:|:-------:|:------------:|:----:|:------------:|
|    Batched Queue    |  ephemeral  |      yes     |  O(1) |   O(1)  | O(n) / O(1)* | O(1) | O(n) / O(1)* |
|    Banker's Queue   |  persistent |      yes     |  O(1) |   O(1)  | O(n) / O(1)* | O(1) | O(n) / O(1)* |
|  Physicist's Queue  |  persistent |      yes     |  O(1) |   O(1)  | O(n) / O(1)* | O(1) | O(n) / O(1)* |
|   Real-Time Queue   |  persistent |      no      |  O(1) |   O(1)  |     O(1)     | O(1) |     O(1)     |
| Hood-Melville Queue |  persistent |      no      |  O(1) |   O(1)  |     O(1)     | O(1) |     O(1)     |

** amortized time*

In [ ]:
class Queue q where
    
    -- | Construct new (empty) queue
    empty :: q a
    
    -- | Check whether a queue is empty
    isEmpty :: q a -> Bool
    
    -- | Append new item to the back of a queue
    snoc :: q a -> a -> q a
    
    -- | Retrieve the head item of a non-empty queue
    head :: q a -> a
    
    -- | Remove the head of a non-empty queue and retrieve the rear
    tail :: q a -> q a

### Batched Queue

In [ ]:
data BatchedQueue a = BQ [a] [a]

-- | Helper function that maintains the 'BatchedQueue' invariant:
-- | *A queue is empty iff the front part is empty*
-- | 
-- | This invariant is preserved by reversing the rear and replacing the front
-- | whenever the front is empty.
checkf :: ([a], [a]) -> ([a], [a])
checkf ([], r) = (reverse r, [])
checkf q = q

instance Queue BatchedQueue where
    
    -- | Constructs new queue in O(1)
    empty = BQ [] []

    -- | Checks whether the queue is empty in O(1) steps.
    -- | Note: This queue maintains the invariant that if the front part is empty, the queue is empty
    -- |       check the 'checkf' helper function.
    isEmpty (BQ f _) = null f
    
    -- | Appends new element to the back of the queue.
    -- | Runs in O(n), amortized time with 'tail' is O(1).
    snoc (BQ f r) x = let (f', r') = checkf(f, x : r) in BQ f' r'
    
    -- | Retrieves the head of the queue in O(1)
    head (BQ [] _) = error "Queue is empty"
    head (BQ (x:_) _) = x
    
    -- | Removes the head of the queue and returns the rest.
    -- | Runs in O(n), amortized time with 'snoc' is O(1).
    tail (BQ [] _) = error "Queue is empty"
    tail (BQ (x:f) r) = let (f', r') = checkf (f, r) in BQ f' r'

### Banker's Queue
`BankersQueue` is a persistent version of the `BatchedQueue` that is based on the *banker's method*.

In [ ]:
data BankersQueue a = BQ Int [a] Int [a]

-- | Rotates the queue if |r| > |f|.
check :: Int -> [a] -> Int -> [a] -> BankersQueue a
check lenf f lenr r =
    if lenr <= lenf then BQ lenf f lenr r
    else BQ (lenf + lenr) (f ++ reverse r) 0 []

instance Queue BankersQueue where

    -- | Constructs new queue in O(1)
    empty = BQ 0 [] 0 []

    -- | Checks whether the queue is empty in O(1) steps.
    isEmpty (BQ lenf _ _ _) = lenf == 0
    
    -- | Appends new element to the back of the queue.
    -- | Runs in O(n), amortized time with 'tail' is O(1).
    snoc (BQ lenf f lenr r) x = check lenf f (lenr + 1) (x:r)
    
    -- | Retrieves the head of the queue in O(1)
    head (BQ _ [] _ _) = error "Queue is empty"
    head (BQ _ (x:_) _ _) = x
    
    -- | Removes the head of the queue and returns the rest.
    -- | Runs in O(n), amortized time with 'snoc' is O(1).
    tail (BQ _ [] _ _) = error "Queue is empty"
    tail (BQ lenf (x:f) lenr r) = check (lenf - 1) f lenr r

### Physicist's Queue
`PhysicistsQueue` is a persistent version of the `BatchedQueue` that is based on the *physicist's method*.

In [ ]:
data PhysicistsQueue a = PQ [a] Int [a] Int [a]

-- | Rotates the queue if |r| > |f| and constructs the queue via 'checkw'.
check :: [a] -> Int -> [a] -> Int -> [a] -> PhysicistsQueue a
check w lenf f lenr r =
    if lenr <= lenf then checkw w lenf f lenr r
    else checkw f (lenf + lenr) (f ++ reverse r) 0 []

-- | Checks the and possibly updates the front prefix (the working list).
checkw :: [a] -> Int -> [a] -> Int -> [a] -> PhysicistsQueue a
checkw [] lenf f lenr r = PQ f lenf f lenr r
checkw w lenf f lenr r = PQ w lenf f lenr r

instance Queue PhysicistsQueue where
    -- | Constructs new queue in O(1)
    empty = PQ [] 0 [] 0 []

    -- | Checks whether the queue is empty in O(1) steps.
    isEmpty (PQ _ lenf _ _ _) = lenf == 0
    
    -- | Appends new element to the back of the queue.
    -- | Runs in O(n), amortized time with 'tail' is O(1).
    snoc (PQ w lenf f lenr r) x = check w lenf f (lenr + 1) (x:r)
    
    -- | Retrieves the head of the queue in O(1)
    head (PQ [] _ _ _ _) = error "Queue is empty"
    head (PQ (x:_) _ _ _ _) = x
    
    -- | Removes the head of the queue and returns the rest.
    -- | Runs in O(n), amortized time with 'snoc' is O(1).
    tail (PQ [] _ _ _ _) = error "Queue is empty"
    tail (PQ (x:w) lenf f lenr r) = check w (lenf - 1) (Prelude.tail f) lenr r

### Real-Time Queue
`RealTimeQueue` eliminates the amortization of the other persistent queue instances and thus has all operations $O(1)$ in the worst case.

This queue implementation incorporates *lazy rebuilding* to make rotation incremental and *scheduling* to ensure the efficiency of rotation (see below) and thus is an instance of a *global rebuilding* data structure.

It is the simplest and fastest queue implementation for applications utilizing persistence.

#### Representation
The queue data structure has three components
 1. the front of the queue
 1. the (reversed) rear of the queue
 1. a *schedule* containing suspensions constructing nodes of the front that will be forced at specific (scheduled) time later on

#### Rotation
The idea here is to break the monolithic rotation of items from the rear list to the front and make it incremental. 

The rotation of a `BankersQueue` consists of two operations: `(++)` and `reverse`. To make the rotation incremental we execute one step of the reversal for every step of `++`. Our new `rotate` function has an extra *accumulating parameter* which is used to accumulate the partial results of `reverse`.

The rotation occurs when $|r| = |f| + 1$ - this is maintained as an invariant.

#### Scheduling
Members of queue's *schedule* are carefully evaluated via `exec` so that all the front dependencies (the `xs` list) are forced (and thus memoized) prior to the call to `rotate` for respective front list. This in turn make the `rotate` run in constant time, creating just suspensions.

In [ ]:
-- | RTQ = (front, rear, schedule)
data RealTimeQueue a = RTQ [a] [a] [a]

-- | Lazily rotate the rear to the front via scheduled suspension.
-- |
-- | This function performs both '(f++)' and 'reverse r' incrementally and
-- | in a lazy fastion. The treatment of schedule guarantees that every node
-- | in 'xs' was forced and memoized prior to the rotation.
-- |
-- | Hence the O(1) execution of every suspension in 'exec'.
rotate :: [a] -> [a] -> [a] -> [a]
rotate [] (y:_) a = y:a
rotate (x:xs) (y:ys) a = x : rotate xs ys (y:a)

-- | Executes a suspension from the schedule.
-- |
-- | When the schedule is non-empty we just pop out its head.
-- |
-- | Otherwise we rotate the queue to produce new front and yield new queue
-- | with it as new front and schedule.
-- |
-- | Since schedule equals new front after a rotation and the next rotation
-- | happens when we force all suspensions from the schedule, 'exec' maintains
-- | the invariant that |s| = |f| - |r| which corresponds to |f| >= |r|.
-- |
-- | Note: Pattern matching against 's' forces and memoizes the next suspension.
exec :: [a] -> [a] -> [a] -> RealTimeQueue a
exec f r (_:s) = RTQ f r s
exec f r [] = let f' = rotate f r [] in RTQ f' [] f'

instance Queue RealTimeQueue where

    -- | Constructs new queue in O(1)
    empty = RTQ [] [] []

    -- | Checks whether the queue is empty in O(1) steps.
    isEmpty (RTQ [] _ _) = True
    isEmpty _ = False
    
    -- | Appends new element to the back of the queue in O(1) worst case time.
    -- |
    -- | New item is added to the rear and a suspension is executed from the schedule.
    snoc (RTQ f r s) x = exec f (x:r) s
    
    -- | Retrieves the head of the queue in O(1)
    head (RTQ [] _ _) = error "Queue is empty"
    head (RTQ (x:_) _ _) = x
    
    -- | Removes the head of the queue and returns the rest in O(1) worst case time.
    -- |
    -- | Head of the front is popped out and a suspension is executed from the schedule and
    -- | with the rest of the front list.
    tail (RTQ [] _ _) = error "Queue is empty"
    tail (RTQ (x:f) r s) = exec f r s

### Hood-Melville Real-Time Queue
`HoodMelvilleQueue` is a real-time queue similar to the `RealTimeQueue` (i.e. persistent with worst case constant operations).

The difference is that this instance is a closer representant of the *global rebuilding* technique while the `RealTimeQueue` showcases the *lazy rebuilding* and *scheduling* combination to achieve the same properties. Particularly, the differences lie in the details of the incremental rotation and consequently the queue representation.

#### Rotation
Again the goal is to make reversal and then append plus reversal incremental.

First an incremental reversal can be captured by two states: `Working xs ys` and `Done xs` where the former represents gradual transfer of the head of `xs` to `ys` and the letter the final state.

Appending two lists `xs` and `ys` works similarly in two passes:
 1. reverse `xs` to get `xs'` - state `Reversing xs xs' ys`
 1. reverse `xs'` onto `ys` - state `Appending xs' ys`

Finally, to append `f` onto `reverse r` we need three reversals in total, two  of which can be done in parallel:
 1. reverse `f` and `r` in parallel to get `f'` and `r'` - state `Reversing f f' r r'`
 1. reverse `f'` onto `r'` - state `Appending f' r'`

There is, however, a problem with the state transitioning if done in a naive way. If we call `tail` during a rotation $k$ times then the first $k$ elements of the resulting list are invalid. To fix this we have few options:
 1. Track the number of invalid elements and add new rotational state `Deleting` that removes requred number of invalid elements (this corresponds to the definition of *global rebuilding*)
 1. Track the number `ok` of valid elements in `f'` and stop the rotation from `f'` ot `r'` when `ok == 0`. Every call to `tail` during a rotation then `invalidate`s elements (decrements `ok`).

#### Representation
First the `RotationState` consists of four states:
 - `Reversing`, `Appending` and `Done` correspond to the states described above
 - `Idle` is a dummy state that is used for situations "inbetween" rotations

Then the queue itself is a five-tuple consisting of
 1. field `lenf` is the length of the list that is under construction (not the working copy - the front field)
 1. next field is a working copy of the front of the queue (an old copy of the front that is under construction to efficiently answer `head` queries)
 1. then there's the `RotationState` described above
 1. field `lenr` is the length of the rear
 1. the last field is the rear of the queue

During a rotation the head elements of the queue end up in the back of `f'` within the rotational state. By keeping a working copy of the old front in the queue we can efficiently answer `head` queries. This is sound as long as new copy is ready by the time the old is exhausted.

In [ ]:
data RotationState a = Idle -- | dummy state
                       -- | (no. valid, f, f', r, r') - moving f onto f' and r onto r'
                       | Reversing Int [a] [a] [a] [a]
                       -- | (no. valid, f', r') - moving f' onto r'
                       | Appending Int [a] [a]
                       -- | final result
                       | Done [a]

-- | HMQ = (len of f' in rotational state, old front, rotational state, len of rear, rear)
data HoodMelvilleQueue a = HMQ Int [a] (RotationState a) Int [a]

-- | Rotational state transition function (runs in constant time).
-- |  - Keep reversing until the front is exhausted (increment no. valid elements)
-- |  - Keep moving elements from f' to r' until f' is exhausted (decrements no. valid elements)
-- |  - Idle and Done states are terminal
exec :: RotationState a -> RotationState a
exec (Reversing ok (x:f) f' (y:r) r') = Reversing (ok + 1) f (x:f') r (y:r')
exec (Reversing ok [] f' [y] r') = Appending ok f' (y:r')
exec (Appending 0 _ r') = Done r'
exec (Appending ok (x:f') r') = Appending (ok - 1) f' (x:r')
exec state = state

-- | State transition function that removes invalid elements (runs in constant time).
-- |  - Decrements no. valid elements in Reversal state and Appending state with some valid elements
-- |  - Terminates appending state when there are no valid elements
-- |  - Noop for other states
invalidate :: RotationState a -> RotationState a
invalidate (Reversing ok f f' r r') = Reversing (ok - 1) f f' r r'
invalidate (Appending 0 _ (_:r')) = Done r'
invalidate (Appending ok f' r') = Appending (ok - 1) f' r'
invalidate state = state

-- | Make two rotational transitions and return resulting queue.
-- |
-- | Since 'exec' runs in O(1) worst case time so does 'exec'. Also by executing two transitions
-- | 'exec2' ensures that the rotation finishes in time to replace the working copy.
exec2 :: Int -> [a] -> RotationState a -> Int -> [a] -> HoodMelvilleQueue a
exec2 lenf f state lenr r = case exec $ exec state of
    (Done newf) -> HMQ lenf newf Idle lenr r
    newstate -> HMQ lenf f newstate lenr r

-- | Run partial rotation and return new queue.
-- |  - When |r| = |f| + 1 new rotation is started and partially executed via 'exec2'
-- |  - Otherwise we just run two rotational steps from the current state
-- |
-- | Runs in O(1) worst case for the same reason as 'exec2' does.
-- |
-- | Note: When new rotaiton starts, the resulting queue front is a workig copy of the old f.
check :: Int -> [a] -> RotationState a -> Int -> [a] -> HoodMelvilleQueue a
check lenf f state lenr r =
    if lenr <= lenf then exec2 lenf f state lenr r
    else let newstate = Reversing 0 f [] r [] in exec2 (lenf + lenr) f newstate 0 []

instance Queue HoodMelvilleQueue where

    -- | Constructs new queue in O(1)
    empty = HMQ 0 [] Idle 0 []

    -- | Checks whether the queue is empty in O(1) steps.
    isEmpty (HMQ lenf _ _ _ _) = lenf == 0
    
    -- | Appends new element to the back of the queue in O(1) worst case time.
    snoc (HMQ lenf f state lenr r) x = check lenf f state (lenr + 1) (x:r)
    
    -- | Retrieves the head of the queue in O(1) worst case time.
    -- |
    -- | Note: Since the front is a working copy of the old front we can just peek its head.
    -- |       Moreover, this is sound because 'snoc' and 'tail' ensure (via 'exec2') that the
    -- |       new copy replaces the front before it is ever exhausted.
    head (HMQ _ [] _ _ _) = error "Queue is empty"
    head (HMQ _ (x:_) _ _ _) = x
    
    -- | Removes the head of the queue and returns the rest in O(1) worst case time.
    -- |
    -- | Note: Calls to 'tail' must invalidate elements in case the state is an ongoing rotation.
    tail (HMQ _ [] _ _ _) = error "Queue is empty"
    tail (HMQ lenf (x:f) state lenr r) = check (lenf - 1) f (invalidate state) lenr r

## Double-Ended Queues

#### Overview
|                instance               | persistence | amortization | empty | isEmpty |     cons     | head |     tail     |     snoc     | last |     init     |
|:-------------------------------------:|:-----------:|:------------:|:-----:|:-------:|:------------:|:----:|:------------:|:------------:|:----:|:------------:|
|    Output-Restricted Banker's Deque   |  persistent |      yes     |  O(1) |   O(1)  | O(n) / O(1)* | O(1) | O(n) / O(1)* | O(n) / O(1)* |   -  |       -      |
|   Output-Restricted Real-Time Deque   |  persistent |      no      |  O(1) |   O(1)  |     O(1)     | O(1) |     O(1)     |     O(1)     |   -  |       -      |
| Output-Restricted Hood-Melville Deque |  persistent |      no      |  O(1) |   O(1)  |    O(1)**    | O(1) |     O(1)     |     O(1)     |   -  |       -      |
|             Banker's Deque            |  persistent |      yes     |  O(1) |   O(1)  | O(n) / O(1)* | O(1) | O(n) / O(1)* | O(n) / O(1)* | O(1) | O(n) / O(1)* |       -      |

*** via the `ConsQueue` wrapper*

#### Qutput-Restricted Deques
An intermediate step between FIFO queues and *double-ended queue (or deques)* are *output-restricted queues* which, additionaly to ordinary queues, provide a `cons :: a -> q a -> q a` function. Class `OutputRestrictedDeque` thus supports insertion to both ends but inspection and removal only from the front.

Some `Queue` instances can easily be extended with custom `cons` implementations. For instance for the Banker's queue we have
```haskell
cons :: a -> BankersQueue a -> BankersQueue a
cons x (BQ lenf f lenr r) = BQ (lenf + 1) (x:f) lenr r
```

Similarly, the real-time queue prepends new element `x` to `f` but also to `s` to maintain the invariant $|s| = |f| - |r|$:
```haskell
cons :: a -> RealTimeQueue a -> RealTimeQueue a
cons x (RTQ f r s) = RTQ (x:f) r (x:s)
```

On the other hand, for some queues (e.g. the `HoodMelvilleQueue`) such trivial implementation is not possible. Fortunately, it's possible to implement generic wrapper (see `ConstQueue` below) which maintains explicit front list and provides `cons` in $O(1)$ worst case time and other operations with constant overhead over respective operations of an underlying queue.

In [ ]:
{-# LANGUAGE FlexibleInstances, FlexibleContexts #-}

class OutputRestrictedDeque q where
    
    -- | Construct new (empty) deque
    empty :: q a
    
    -- | Check whether a deque is empty
    isEmpty :: q a -> Bool
    
    -- Insert, inspect and remove the front element
    cons :: a -> q a -> q a
    head :: q a -> a
    tail :: q a -> q a
    
    -- | Append new item to the back of a queue
    snoc :: q a -> a -> q a


data ConsQueue q a = ConsQueue [a] (q a)

instance Queue q => OutputRestrictedDeque (ConsQueue q) where

    -- | Constructs new queue in O(empty) time for 'empty' of the underlying queue
    empty = ConsQueue [] Ghci2.empty

    -- | Checks whether the queue is empty in O(isEmpty q) time.
    isEmpty (ConsQueue f q) = null f && Ghci2.isEmpty q
    
    -- Prepend new element to the front of the queue in O(1) worst case time.
    cons x (ConsQueue f q) = ConsQueue (x:f) q
    
    -- | Appends new element to the back of the queue in O(snoc q) time.
    snoc (ConsQueue f q) x = ConsQueue f (Ghci2.snoc q x)
    
    -- | Retrieves the head of the queue in O(head q) time.
    head (ConsQueue [] q) = Ghci2.head q
    head (ConsQueue (x:_) _) = x
    
    -- | Removes the head of the queue and returns the rest in O(tail q) time.
    tail (ConsQueue [] q) = ConsQueue [] (Ghci2.tail q)
    tail (ConsQueue (_:f) q) = ConsQueue f q

#### Deques
The `Deque` class provides on top of standard `Queue` operations and `cons` two additional operations:
 - `last` peeks the last element of the deque
 - `init` removes the last element of the deque (i.e. returns a queue with all items but the last one)

In [ ]:
class Deque q where

    -- | Construct new (empty) deque
    empty :: q a
    
    -- | Check whether a deque is empty
    isEmpty :: q a -> Bool
    
    -- Insert, inspect and remove the front element
    cons :: a -> q a -> q a
    head :: q a -> a
    tail :: q a -> q a
    
    -- Insert, inspect and remove the rear element
    snoc :: q a -> a -> q a
    last :: q a -> a
    init :: q a -> q a

### Banker's Deques
While standard queues can be berfetly balanced (all elemnts end up in the front part), this is not possible for deques that can be updated from both ends which define perfect balance as having equal distribution of elements between the front and the rear part.

The idea behind Banker's Deque is that a balanced deque maintains follwing invariant (for a constant $c > 0$):
$$
|f| \leq c|r| + 1 \land |r| \leq c|f| + 1
$$
This invariant ensures that any deque is no more off perfect balance than by a factor of $c$. Moreover, it works for singleton deques with the single element being in either the front or the rear part due to the $+1$.

#### Representation
`BankersDeque` has the same representation as the `BankersQueue`, that is
 - length of the front part
 - the front list
 - length of the rear part
 - the rear list

*Note: It might be possible to parametrize the data structure with $c$ but we keep the implementation simple.*

#### Balancing
The helper function `check` is responsible for maintaining the balancing invariant. More precisely it checks whether one of the lists is too long and if so it transfers the items onto the back of the other.

*Note: More on the amortized complexity analysis can be found in the book.*
#### Reversal
As a bonus there's the `rev` function which takes advantage of the symmetry in the implementation of `check` and trivially reverses a deque in $O(1)$ time. This is quite common property between deques.

Note that one could implement the update operations on the deque via `rev` but the approach show below is slightly faster. Just as an example: `init = rev . tail . rev`.

In [ ]:
-- | BD = (len of front, front, len of rear, rear)
data BankersDeque a = BD Int [a] Int [a]

-- | Balancing coefficient c > 1
balanceCoef = 3 :: Int

-- | Check and possibly re-balance a deque in O(n) worst case and O(1) amortized time.
-- |
-- | This function maintans the invariant: |f| <= c * |r| + 1 and |r| <= c * |f| + 1.
check :: Int -> [a] -> Int -> [a] -> BankersDeque a
check lenf f lenr r
    | lenf > balanceCoef * lenr + 1 =
        let i = (lenf + lenr) `div` 2
            j = lenf + lenr - i
            f' = take i f
            r' = r ++ reverse (drop i f)
        in BD i f' j r'
    | lenr > balanceCoef * lenf + 1 =
        let j = (lenf + lenr) `div` 2
            i = lenf + lenr - j
            r' = take j r
            f' = f ++ reverse (drop j r)
        in BD i f' j r'
    | otherwise = BD lenf f lenr r

-- | Reverse given deque in O(1) worst case time.
rev :: BankersDeque a -> BankersDeque a
rev (BD lenf f lenr r) = BD lenr r lenf f

instance Deque BankersDeque where

    -- | Construct new (empty) deque in O(1) worst case time.
    empty = BD 0 [] 0 []
    
    -- | Check whether a deque is empty in O(1) worst case time.
    isEmpty (BD lenf _ lenr _) = (lenf + lenr) == 0
    
    -- | Prepend new element to the front of the deque in
    -- | O(n) worst case and O(1) amortized time.
    -- |
    -- | Note: Resulting deque is constructed by 'check' which gives the time complexity.
    cons x q @ (BD lenf f lenr r) = check (lenf+1) (x:f) lenr r
    
    -- | Peek the front element of the deque in O(1) worst case time.
    head (BD _ [] _ _) = error "Deque is empty"
    head (BD _ (x:_) _ _) = x
    
    -- | Remove the front element from a non-empty deque in 
    -- | O(n) worst case and O(1) amortized time.
    -- |
    -- | Note: Resulting deque is constructed by 'check' which gives the time complexity.
    tail (BD _ [] _ _) = error "Deque is empty"
    tail (BD lenf (x:f') lenr r) = check (lenf-1) f' lenr r
    
    -- | Append new element to the back of the deque in
    -- | O(n) worst case and O(1) amortized time.
    -- |
    -- | Note: Resulting deque is constructed by 'check' which gives the time complexity.
    snoc (BD lenf f lenr r) x = check lenf f (lenr+1) (x:r)
    
    -- | Peek the rear element of the deque in O(1) worst case time.
    last (BD _ _ _ []) = error "Deque is empty"
    last (BD _ _ _ (x:_)) = x
    
    -- | Remove the rear element from a non-empty deque in
    -- | O(n) worst case and O(1) amortized time.
    -- |
    -- | Note: Resulting deque is constructed by 'check' which gives the time complexity.
    init (BD _ _ _ []) = error "Deque is empty"
    init (BD lenf f lenr (x:r')) = check lenf f (lenr-1) r'

## Heaps

#### Overview
|         instance        | persistence | amortization | empty | isEmpty |       insert      |       merge       |           findMin          |     deleteMin     |
|:-----------------------:|:-----------:|:------------:|:-----:|:-------:|:-----------------:|:-----------------:|:--------------------------:|:-----------------:|
|       Leftist Heap      |  ephemeral  |      no      |  O(1) |   O(1)  |     O(log(n))     |     O(log(n))     |            O(1)            |     O(log(n))     |
|      Binomial Heap      |  persistent |      yes     |  O(1) |   O(1)  | O(log(n)) / O(1)* |     O(log(n))     |     O(log(n)) / O(1)**     |     O(log(n))     |
| Scheduled Binomial Heap |  persistent |      no      |  O(1) |   O(1)  |        O(1)       |     O(log(n))     |     O(log(n)) / O(1)**     |     O(log(n))     |
|        Splay Heap       |  ephemeral  |      yes     |  O(1) |   O(1)  | O(n) / O(log(n))* | O(n) / O(log(n))* | O(n) / O(log(n))* / O(1)** | O(n) / O(log(n))* |
|       Pairing Heap      |  ephemeral  |      yes     |  O(1) |   O(1)  |        O(1)       |        O(1)       |            O(1)            | O(n) / O(log(n))* |
|    Lazy Pairing Heap    |  persistent |      yes     |  O(1) |   O(1)  |        TODO       |        TODO       |            O(1)            |        TODO       |

** amortized time*

*** with explicit reference to the minimum element*

#### ExplicitMin Heap
`ExplicitMinHeap` is a `Heap` wrapper that explicitly tracks the minimum element of the underlying heap to make the `findMin` access $O(1)$.

Updating the reference to the minimum element takes $O(1)$ time and thus `insert`, `merge` and `deleteMin` have their complexities unchanged (resp. the time complexity of these for the `ExplicitMinHeap` is the same as of the underlying `Heap`).

Tracking the min. element in `deleteMin` involves call to the underlying `findMin` (to lookup new minimum). In case the original `findMin` was slower than `deleteMin`, the performance might degrade. If this is a problem, don't use `ExplicitMinHeap` wrapper and rather update the underlying instance itself.

In [ ]:
class Heap h where

    -- | Construct new (empty) heap
    empty :: Ord a => h a
    
    -- | Check whether a heap is empty
    isEmpty :: h a -> Bool
    
    -- | Add new item to a heap
    insert :: Ord a => a -> h a -> h a
    
    -- | Merge two heaps together
    merge :: Ord a => h a -> h a -> h a
    
    -- | Retrieve the minimum element of a heap
    findMin :: Ord a => h a -> a
    
    -- | Remove the minimum element of a heap and return resulting heap
    deleteMin :: Ord a => h a -> h a


data ExplicitMinHeap h a = Empty | ExplicitMinHeap a (h a)

instance Heap h => Heap (ExplicitMinHeap h) where

    -- | Construct an empty heap in O(1).
    empty = Empty
    
    -- | Check whether given heap is empty.
    isEmpty Empty = True
    isEmpty (ExplicitMinHeap _ h) = isEmpty h && error "Non-empty explicit-min heap with an empty underlying heap"
    
    -- | Add new item to the heap and update the minimum.
    -- |
    -- | Note: Tracking min. element adds only constant overhead and thus does not affect
    -- |       the overall complexity of 'insert'.
    insert x Empty = ExplicitMinHeap x (insert x empty)
    insert x (ExplicitMinHeap y h) = ExplicitMinHeap (min x y) (insert x h)
    
    -- | Merge two heaps together and update the minimum.
    -- |
    -- | Note: Tracking min. element adds only constant overhead and thus does not affect
    -- |       the overall complexity of 'merge'.
    merge (ExplicitMinHeap x h) Empty = ExplicitMinHeap x (merge h empty)
    merge Empty (ExplicitMinHeap x h) = ExplicitMinHeap x (merge empty h)
    merge (ExplicitMinHeap x h1) (ExplicitMinHeap y h2) = ExplicitMinHeap (min x y) (merge h1 h2)
    
    -- | Retrieve the minimum element of a heap in O(1) time.
    -- |
    -- | Note: This improves the complexity of 'findMin' for the base heap to constant.
    findMin Empty = error "Heap is empty"
    findMin (ExplicitMinHeap x _) = x
    
    -- | Remove the minimum element of the underlying heap and lookup new minimum.
    -- |
    -- | Note: Although we do a lookup of new min. element, this does not increase
    -- |       the overall complexity of 'deleteMin' of the underlying heap unless
    -- |       this operation was faster than original 'findMin'.
    deleteMin Empty = error "Heap is empty"
    deleteMin (ExplicitMinHeap _ h) = let h' = deleteMin h in ExplicitMinHeap (findMin h') h'

### Leftist Heap
This implementation is based on a [Leftist tree](https://en.wikipedia.org/wiki/Leftist_tree) without any fancy optimization techniques. Notably, in this implementation `merge` takes two passes:
 1. top-down pass consisting of calls to `merge`
 1. bottom-up pass consisting of calls to `makeNode`

In [ ]:
data LeftistHeap a = Empty | Node Int a (LeftistHeap a) (LeftistHeap a)

-- | Extract the rank of a heap node (zero for empty heap)
rank :: LeftistHeap a -> Int
rank Empty = 0
rank (Node r _ _ _) = r

-- | Create new heap node with given sub-trees while maintaining that
-- | the rank of the left sub-tree is at least as large as the right one.
makeNode :: Ord a => a -> LeftistHeap a -> LeftistHeap a -> LeftistHeap a
makeNode x a b = if ra >= rb then Node (rb + 1) x a b else Node (ra + 1) x b a
    where
        ra = rank a
        rb = rank b

instance Heap LeftistHeap where

    -- | Construct an empty heap in O(1).
    empty = Empty
    
    -- | Check whether given heap is empty in O(1).
    isEmpty Empty = True
    isEmpty _ = False
    
    -- | Add new item to the heap.
    -- |
    -- | The item is first turned to a trivial heap and merged into.
    -- | Therefore this call runs in O(log(n)) steps in the worst case.
    insert x h = merge (Node 1 x Empty Empty) h
    
    -- | Merge two heaps together in O(log(n)) steps.
    -- | Note: The tree is kept balanced because 'makeNode' balances sub-trees via 'rank'.
    merge h Empty = h
    merge Empty h = h
    merge h1 @ (Node _ x a1 b1) h2 @ (Node _ y a2 b2) = 
        if x <= y then makeNode x a1 (merge b1 h2)
        else makeNode y a2 (merge h1 b2)
    
    -- | Retrieve the minimum element of a heap in O(1) worst case time.
    findMin Empty = error "Heap is empty"
    findMin (Node _ x _ _) = x
    
    -- | Remove the minimum element of a heap.
    -- | 
    -- | Becasue the minimum element is the root, this funciton just calls 'merge' on
    -- | both root sub-trees and thus runs in O(log(n)) worst case time.
    deleteMin Empty = error "Heap is empty"
    deleteMin (Node _ x a b) = merge a b

### Binomial Heap
This implementation is based on a set (forest) of binomial trees described for instance [here](https://en.wikipedia.org/wiki/Binomial_heap).

There are no fancy optimization techniques, for instance the heap described below does not track the minimum element.

#### Benefits of Binomial Heap
 - Compared to standard *Binary Heap* supports merging (melding) in O(log(n)) time
 - Compared to *Leaftist Heap* has O(1) amortized inserts and merging is based on the size of the larger heap

#### Representation
A `BinomialHeap` is a collection of heap-ordered `BinomialTree`s in ascending order of their ranks in which no two trees have the same rank.

A binomial tree is defined recursively as
 - A binomial tree of rank $0$ is a singleton node
 - A binomial tree of rank $r + 1$ if formed by *linking* two binomial trees of rank $r$, making one tree the leftmost child of the other

Alternatively a binomial tree of rank $r$ is a node with $r$ children $t_1, \dots, t_r$ where each $t_i$ is a binomial tree of rank $r - i$ (i.e. the list of children is stored in decreasing order of rank in each `Node`).

Note that we explicitly store rank in each `Node` even though it is not strictly necessary and different representations are also possible. See for instance the one used by the *Scheduled Binomial Heap* which exploits the fact that a binomial tree of rank $r$ contains $2^r$ elements and thus trees in a heap of size $n$ correspond exactly to the 1s in the binary representation of $n$.

From the binary representation follows the observation that there can only be $O(log(n))$ binomial trees in a heap of size $n$ as there can only be that many 1s in the binary representation.

In [ ]:
data BinomialTree a = Node Int a [BinomialTree a]

newtype BinomialHeap a = BinomialHeap [BinomialTree a]

-- | Extract the rank of a binomial tree
rank :: BinomialTree a -> Int
rank (Node r _ _) = r

-- | Extract the root element of a binomial tree
root :: BinomialTree a -> a
root (Node _ x _) = x

-- | Link two binmoial trees of the same rank together in O(1) time.
--
-- | Tree having the greater root is added as the leftmost child of of the other.
-- | Resulting tree has rank incremented by one.
link :: Ord a => BinomialTree a -> BinomialTree a -> BinomialTree a
link t1 @ (Node r x1 c1) t2 @ (Node _ x2 c2) =
    if x1 <= x2 then Node (r + 1) x1 (t2 : c1)
    else Node (r + 1) x2 (t1 : c2)

-- | Insert a tree in given heap.
-- |
-- | Since there are at most O(log(n)) trees, 'link' takes O(1) and in the worst case we traverse
-- | all the trees, the time complexity is logarithmic.
insTree :: Ord a => BinomialTree a -> BinomialHeap a -> BinomialHeap a
insTree t (BinomialHeap []) = BinomialHeap [t]
insTree t (BinomialHeap ts @ (t' : ts')) =
    -- if the tree rank is not present in the heap then just add the tree
    if rank t < rank t' then BinomialHeap (t:ts)
    -- otherwise fold over the heap trees while linking them
    else insTree (link t t') (BinomialHeap ts')

-- | Finds and removes a tree with the minimum element from a heap.
-- |
-- | This function returns both the minimum tree and resulting heap and runs in logarithmic worst case time
-- | as there are only O(log(n)) trees in any heap and we need to check only roots.
removeMinTree :: Ord a => BinomialHeap a -> (BinomialTree a, BinomialHeap a)
removeMinTree (BinomialHeap []) = error "Heap is empty"
removeMinTree (BinomialHeap [t]) = (t, empty)
removeMinTree (BinomialHeap (t:ts)) = let (t', (BinomialHeap ts')) = removeMinTree (BinomialHeap ts) in
    if root t <= root t' then (t, BinomialHeap ts)
    else (t', BinomialHeap (t:ts'))

instance Heap BinomialHeap where
    
    -- | Construct an empty heap in O(1).
    empty = BinomialHeap []
    
    -- | Check whether given heap is empty in O(1).
    isEmpty (BinomialHeap ts) = null ts
    
    -- | Add new item to the heap.
    -- |
    -- | The item is first turned to a trivial binomial tree of rank 0 and inserted into
    -- | the heap via 'insTree'.
    -- |
    -- | The worst case time complexity is O(log(n)) but can be amortized by other updates to just O(1).
    insert x h = insTree (Node 0 x []) h
    
    -- | Merge two heaps together in O(log(n)) steps.
    merge h (BinomialHeap []) = h
    merge (BinomialHeap []) h = h
    merge h1 @ (BinomialHeap (t1:ts1')) h2 @ (BinomialHeap (t2:ts2')) =
        case compare (rank t1) (rank t2) of
            LT -> let (BinomialHeap ts) = merge (BinomialHeap ts1') h2 in BinomialHeap (t1:ts)
            GT -> let (BinomialHeap ts) = merge h1 (BinomialHeap ts2') in BinomialHeap (t2:ts)
            EQ -> insTree (link t1 t2) (merge (BinomialHeap ts1') (BinomialHeap ts2'))
    
    -- | Retrieve the minimum element of a heap in O(log(n)) worst case time.
    -- | Note: The time complexity follows from the analysis of 'removeMinTree'.
    findMin h = let (t, _) = removeMinTree h in root t
    
    -- | Remove the minimum element of a heap in O(log(n)) worst case time.
    -- | Note: The time complexity follows from 'removeMinTree' and 'merge' and the fact that
    -- |       there is at most logarithmic number of trees in a heap (for 'reverse').
    deleteMin h = let ((Node _ _ ts1), h') = removeMinTree h in merge (BinomialHeap (reverse ts1)) h'

### Scheduled Binomial Heap
`ScheduledBinomialHeap` is a persistent variant of the *Binomial Heap* that uses *lazy rebuilding* in combination with *scheduling* to make `insert` run in $O(1)$ worst case time. Other operations on the heap retain their worst case bounds.

#### Insertion

Compared to the `BinomialHeap` implementation, here `insTree` is fully incremental (rather than monolithic). Each job in a schedule then represents all the unevaluated suspentions from single call to `insert`.

Because of how `insert` manages the schedule (via `exec`), all required suspensions are memoized prior to calls to `insTree` which makes the whole insertion run in worst case constant time.

#### Binomial Tree Representation
In order to make `insTree` incremental, we make an explicit connection between the binomial heaps and binary numbers:
> Trees in a heap correspond to the 1s in the binary representation of the size of the heap.

This way we can
 - create an explicit binary represenation of binomial trees in a heap via `[Digit a]` and eliminate the rank field of the `Node` constructor since a tree in the $i$-th digit has rank $i$ and the children of a rank $r$ node have ranks $r - 1, \dots, 0$
 - make intermediate steps of `insTree` return a suspension with the `Zero` constructor and the last completion step with `One tree`

In [ ]:
data BinomialTree a = Node a [BinomialTree a]

-- | Binary representation of a binomial tree in a heap.
-- |
-- | Additionally, 'Zero's represent intermediate steps of 'insTree' while 'One t' represents a
-- | completed insertion of a 't :: BinomialTree a'.
data Digit a = Zero | One (BinomialTree a)

-- | Suspension schedule in which each job '[Digit a]' represents one partial execution of 'insTree'.
type Schedule a = [[Digit a]]

-- | Final heap is a product of a list of binomial trees in binary representation and a schedule
data ScheduledBinomialHeap a = SBH [Digit a] (Schedule a)

-- | Link two binmoial trees of the same rank together in O(1) time.
--
-- | Tree having the greater root is added as the leftmost child of of the other.
link :: Ord a => BinomialTree a -> BinomialTree a -> BinomialTree a
link t1 @ (Node x1 c1) t2 @ (Node x2 c2) = if x1 <= x2 then Node x1 (t2 : c1) else Node x2 (t1 : c2)

-- | Insert a tree in given heap in constant time.
-- |
-- | Contrary to the implementation used by 'BinomialHeap' this funciton is incremental rather than
-- | monlithic (i.e. each case returns a suspension).
-- |
-- | Moreover, due to the interation between this function and 'exec' in 'insert' all scheduled
-- | suspensions that a tree being matched depens on have been evaluated (and thus memoized)
-- | prior to any call to 'insTree', this function runs in O(1) worst case time.
insTree :: Ord a => BinomialTree a -> [Digit a] -> [Digit a]
insTree t [] = (One t) : []
insTree t (Zero : ds) = (One t) : ds
insTree t ((One t') : ds) = Zero : (insTree (link t t') ds)

-- | Merge two heaps represented by a collection of binomial trees (resp. digits) together.
mrg :: Ord a => [Digit a] -> [Digit a] -> [Digit a]
mrg ds1 [] = ds1
mrg [] ds2 = ds2
mrg (Zero:ds1) (d:ds2) = d : (mrg ds1 ds2)
mrg (d:ds1) (Zero:ds2) = d : (mrg ds1 ds2)
mrg ((One t1):ds1) ((One t2):ds2) = Zero : (insTree (link t1 t2) (mrg ds1 ds2))

-- | Normalize a heap by executing all suspensions in it.
-- |
-- | Note: Suspensions are executed by matching against them. Becaues there can be at most
-- |       O(log(n)) pending suspensions in any heap, this function runs in logarithic time.
normalize :: [Digit a] -> [Digit a]
normalize [] = []
normalize ds @ (_:ds') = seq (normalize ds') ds

-- | Execute a job (suspension) from given schedule in constant time.
-- |
-- | There are two interesting cases:
-- |  1. Since any completed insertion ends with a 'One', when we encounter a 'Zero' we keep
-- |     the rest of an unfinished job in the schedule
-- |  2. Otherwise the insertion if done and we remove the job from the schedule
-- |
-- | Note: It can be proven that by the composition of 'insTree' and 'exec' in 'insert' and
-- |       because 'insTree' creates suspensions, this function runs in O(1) worst case time.
exec :: Schedule a -> Schedule a
exec [] = []
exec ((Zero:job) : schedule) = job : schedule
exec (_:schedule) = schedule

-- | Remove the minimum element from given heap and return it alognside the resulting heap.
-- |
-- | This implemntation is just an adaptation to the binary represenation and runs in O(log(n))
-- | worst case time.
-- |
-- | Note: This function may produce streams with trailing 'Zero's. However, these are either
-- |       discarded by 'findMin' or merged with 'One's by 'deleteMin'.
removeMinTree :: Ord a => [Digit a] -> (BinomialTree a, [Digit a])
removeMinTree [] = error "Tree is empty"
removeMinTree [One t] = (t, [])
removeMinTree (Zero:ds) = let (t', ds') = removeMinTree ds in (t', Zero:ds')
removeMinTree ((One t @ (Node x _)):ds) = case removeMinTree ds of
    (t' @ (Node x' _), ds') -> if x <= x' then (t, Zero:ds) else (t', (One t):ds')

instance Heap ScheduledBinomialHeap where
    
    -- | Construct an empty heap in O(1).
    empty = SBH [] []
    
    -- | Check whether given heap is empty in O(1).
    isEmpty (SBH [] _) = True
    isEmpty _ = False
    
    -- | Add new item to the heap in O(1) worst case time.
    -- |
    -- | Note: After inserting new node to the heap we execute two jobs (suspensions) in the scheudle.
    -- |       The amortized cost of 'insTree' used to be 2 so two executions of scheduled suspensions
    -- |       make all suspension dependencies memoized and in consequence 'insTree' can run in O(1).
    insert x (SBH ds schedule) = let ds' = insTree (Node x []) ds in
        SBH ds' $ exec $ exec (ds' : schedule)
    
    -- | Merge two heaps together in O(log(n)) steps in the worst case.
    -- |
    -- | Note: Resulting heap (after merging) is then trivially normalized (i.e. without optimizations)
    -- |       by executing all suspensions in it and then the schedule is reset. Since 'normalize' runs
    -- |       in at most O(log(n)) steps, this does not add any time complexity.
    merge (SBH ds1 _) (SBH ds2 _) = let ds = normalize (mrg ds1 ds2) in SBH ds []
    
    -- | Retrieve the minimum element of a heap in O(log(n)) worst case time.
    -- |
    -- | Note: The time complexity follows from the analysis of 'removeMinTree' and can be imroved
    -- |       to O(1) with explicit tracking via 'ExplicitMinHeap'.
    findMin (SBH ds _) = let ((Node x _), _) = removeMinTree ds in x
    
    -- | Remove the minimum element of a heap in O(log(n)) worst case time.
    -- | 
    -- | The time complexity follows from 'removeMinTree' and 'mrg' and the fact that
    -- | there is at most logarithmic number of trees in a heap (for 'reverse').
    -- |
    -- | Note: Resulting heap (after merging) is then trivially normalized (i.e. without optimizations)
    -- |       by executing all suspensions in it and then the schedule is reset. Since 'normalize' runs
    -- |       in at most O(log(n)) steps, this does not add any time complexity.
    deleteMin (SBH ds _) = SBH (normalize ds'') []
        where
            ((Node x c), ds') = removeMinTree ds
            ds'' = mrg (map One $ reverse c) ds'

### Splay Heap
This `Heap` instance is based on a [Splay Tree](https://en.wikipedia.org/wiki/Splay_tree).

#### Benefits of Splay Heaps
 - Self-balancing (splaying) moves frequently accessed items closer to the root where they can be accessed more quickly in the future
 - One of the fastest heaps when persistence and `merge` are not required (with explicit minimum pointer)
 - Sorting an already sorted array with a `SplayHeap` takes just `O(n)` time
 - Balancing does not require any additional data stored in the tree (resp. nodes)

In [ ]:
data SplayHeap a = Empty | Node (SplayHeap a) a (SplayHeap a)

-- | Splay a pivot element to the root of given Splay tree (heap) and
-- | return a pair of its left sub-tree (smaller than pivot) and the rest (greater than pivot).
-- |
-- | Splaying involves two optimistic balancing operations: Zig-Zig and Zig-Zag and
-- | balances the tree to shorten the leftmost spine (the leftmost element is the minimum).
-- |
-- | Despite splaying, the height of a Splay tree may in the worst case still be linear.
-- | However, the amortized complexity of heap access and update operations is linearithmic.
partition :: Ord a => a -> SplayHeap a -> (SplayHeap a, SplayHeap a)
partition _ Empty = (Empty, Empty)
partition pivot t @ (Node a x b) =
    if x <= pivot then case b of
        Empty -> (t, Empty)
        Node b1 y b2 -> if y <= pivot then let (small, big) = partition pivot b2 in (Node (Node a x b1) y small, big)
                        else let (small, big) = partition pivot b1 in (Node a x small, Node big y b2)
    else case a of
        Empty -> (Empty, t)
        Node a1 y a2 -> if y <= pivot then let (small, big) = partition pivot a2 in (Node a1 y small, Node big x b)
                        else let (small, big) = partition pivot a1 in (small, Node big y (Node a2 x b))

instance Heap SplayHeap where
    
    -- | Construct an empty heap in O(1).
    empty = Empty
    
    -- | Check whether given heap is empty in O(1).
    isEmpty Empty = True
    isEmpty _ = False
    
    -- | Add new item to the heap.
    -- |
    -- | The item is used as a pivot to partition given heap and splayed as new root with
    -- | the smaller elemnts and greater elements as the left and right sub-trees respectively.
    -- |
    -- | As discussed in the 'partition' function, the heap might end up imbalanced, so the
    -- | worst case complexity is O(n) with amortized time complexity O(log(n)).
    insert x h = let (a, b) = partition x h in Node a x b
    
    -- | Merge two heaps together in O(log(n)) amortized steps (O(n) worst case).
    merge Empty h = h
    merge (Node a x b) h = let (ha, hb) = partition x h in Node (merge ha a) x (merge hb b)
    
    -- | Retrieve the minimum element of a heap in O(log(n)) amortized time.
    -- |
    -- | The minimum element in a Splay Tree is the leftmost element so 'findMin' simply
    -- | traverses the leftmost spine in O(log(n)) amortized time.
    -- |
    -- | Note: The time complexity can be reduced to O(1) by wrapping the whole data structure
    -- |       and explicitly tracking a reference to the minimum element.
    findMin Empty = error "Heap is empty"
    findMin (Node Empty x _) = x
    findMin (Node a _ _) = findMin a
    
    -- | Remove the minimum element of a heap.
    -- | 
    -- | The removal procedure is analogous to 'findMin' and with the same argument runs in
    -- | O(log(n)) amortized time.
    deleteMin Empty = error "Heap is empty"
    deleteMin (Node Empty x b) = b
    deleteMin (Node (Node Empty x b) y c) = Node b y c
    deleteMin (Node (Node a x b) y c) = Node (deleteMin a) x (Node b y c)

### Pairing Heap
Implementation of a [Pairing Heap](https://en.wikipedia.org/wiki/Pairing_heap) that does not rely on persistence (i.e. is ephemeral) and does not support `decreaseKey` operation.

#### Benefits of Pairing Heaps
 - Similar performance to Splay heaps but much faster when `merge` is used

In [ ]:
data PairingHeap a = Empty | Node a [PairingHeap a]

-- | Merge a list of heaps into single heap in O(n) worst case and O(log(n)) amortized time.
-- |
-- | Merging is done in two passes:
-- |  1. first merge pairs of heaps in the list (hence the name of the heap)
-- |  2. then reverse the direction and merge heaps by folding from the right
mergePairs :: Ord a => [PairingHeap a] -> PairingHeap a
mergePairs [] = Empty
mergePairs [h] = h
mergePairs (h1:h2:hs) = merge (merge h1 h2) (mergePairs hs)

instance Heap PairingHeap where

    -- | Construct an empty heap in O(1).
    empty = Empty
    
    -- | Check whether given heap is empty in O(1).
    isEmpty Empty = True
    isEmpty _ = False
    
    -- | Add new item to the heap in O(1) worst case time.
    -- |
    -- | Added item is first converted to a trivial heap and then merged into the other heap.
    -- | Therefore the time complexity follows from the analysis of 'merge'.
    insert x h = merge (Node x []) h
    
    -- | Merge two heaps together in O(1) steps.
    -- |
    -- | Merging adds the heap with the greater root as the leftmost sub-tree of the other so that
    -- | the minimum of the two is the root of the result.
    merge h Empty = h
    merge Empty h = h
    merge h1 @ (Node x hs1) h2 @ (Node y hs2) =
        if x <= y then Node x (h2:hs1)
        else Node y (h1:hs2)
    
    -- | Retrieve the minimum element of a heap in O(1) time.
    findMin Empty = error "Heap is empty"
    findMin (Node x _) = x
    
    -- | Remove the minimum element of a heap.
    -- |
    -- | The resulting heap is constructed by ignoring current root (the minimum element) and
    -- | merging sub-trees via 'mergePairs', therefore runs in O(log(n)) amortized time (O(n) worst case).
    deleteMin Empty = error "Heap is empty"
    deleteMin (Node _ hs) = mergePairs hs

### Lazy Pairing Heap
`LazyPairingHeap` is a persistent variant of the `PairingHeap`. The formal analysis of this type of heap is rather involved but it's conjectured to be equivalent to the `PairingHeap` in terms of amortized complexity.

This version is slower for languages with strict evaluation semantics but slowdown is less significant in Haskell where lazy evaluation and memization is default for all data types. Also, the cost of laziness is low for applications that heavily utilize persistence.

In [ ]:
data LazyPairingHeap a = Empty | Node a (LazyPairingHeap a) (LazyPairingHeap a)

-- | Add new child to a node.
-- |
-- | If the odd field is empty then the child's put there. Otherwise, the child
-- | is paired with the child in the odd field.
-- |
-- | This is similar to 'mergePairs' but we are parially forcing suspensions on 'merge',
-- | effectively breaking the monolithic 'deleteMin' into lazy and incremental linking.
link :: Ord a => LazyPairingHeap a -> LazyPairingHeap a -> LazyPairingHeap a
link (Node x Empty m) a = Node x a m
link (Node x b m) a = Node x Empty (merge (merge a b) m)

instance Heap LazyPairingHeap where

    -- | Construct an empty heap in O(1).
    empty = Empty
    
    -- | Check whether given heap is empty in O(1).
    isEmpty Empty = True
    isEmpty _ = False
    
    -- | Add new item to the heap in O(1) steps.
    -- |
    -- | Added item is first converted to a trivial heap and then merged into the other heap.
    insert x h = merge (Node x Empty Empty) h
    
    -- | Merge two heaps together.
    -- |
    -- | This is done using 'link' which adds the greater root as a child to the other heap.
    merge a Empty = a
    merge Empty b = b
    merge a @ (Node x _ _) b @ (Node y _ _) = if x <= y then link a b else link b a
    
    -- | Retrieve the minimum element of a heap in O(1) time.
    findMin Empty = error "Heap is empty"
    findMin (Node x _ _) = x
    
    -- | Remove the minimum element of a heap.
    deleteMin Empty = error "Heap is empty"
    deleteMin (Node _ a m) = merge a m

## Sortable Collections

|       instance       | persistence | amortization | empty |     add    |  sort |
|:--------------------:|:-----------:|:------------:|:-----:|:----------:|:-----:|
|      Merge Sort      |  persistent |      yes     |  O(1) | O(log(n))* | O(n)* |
| Scheduled Merge Sort |  persistent |      no      |  O(1) |  O(log(n)) |  O(n) |

In [ ]:
class Sortable s where
    
    -- | Construct new (empty) collection
    empty :: Ord a => s a
    
    -- | Add new item to an existing sorted collection
    add :: Ord a => a -> s a -> s a
    
    -- | Convert a (sorted) collection into a sorted list
    sort :: Ord a => s a -> [a]

### Bottom-up Merge Sort
An instance of `Sortable` that is based on a [bottom-up *Merge Sort*](https://en.wikipedia.org/wiki/Merge_sort#Bottom-up_implementation).

In [ ]:
data MergeSort a = MS Int [[a]]

-- | Merge two sorted lists together.
mrg :: Ord a => [a] -> [a] -> [a]
mrg [] ys = ys
mrg xs [] = xs
mrg xs @ (x:xs') ys @ (y:ys') =
    if x <= y then x : mrg xs' ys
    else y : mrg xs ys'

instance Sortable MergeSort where

    -- | Construct new (empty) collection in O(1).
    empty = MS 0 []
    
    -- | Add new item to an existing sorted collection in O(log(n)) amortized time.
    add x (MS size segs) = MS (size + 1) (addSeg [x] segs size)
        where
            addSeg seg segs size =
                if even size then seg : segs
                else addSeg (mrg seg (Prelude.head segs)) (Prelude.tail segs) (size `div` 2)
    
    -- | Convert a (sorted) collection into a sorted list in O(n) amortized steps.
    sort (MS size segs) = foldl mrg [] segs

### Scheduled Bottom-up Merge Sort
An instance of `Sortable` that improves on `MergeSort` by incorporating *scheduling* to make the bounds worst case rather than amortized.

In [ ]:
-- | Schedule = [unevaluated suspensions per merge call]
type Schedule a = [[a]]

-- | SMS = (size, [(segment, segment's schedule)])
data ScheduledMergeSort a = SMS Int [([a], Schedule a)]

-- | Execute one merge step from a schedule by matching againt the head of the front stream.
-- | 
-- | Note: Since only the first list in a schedule can ever be empty the second clause, 
-- |       which executes next stream from the schedule if one ends, this function never loops.
exec1 :: Schedule a -> Schedule a
exec1 [] = []
exec1 ([]:schedule) = exec1 schedule
exec1 ((_:xs):schedule) = xs : schedule

-- | Execute the schedule twice and returns resulting schdule paired with the input segment.
exec2 :: [a] -> Schedule a -> ([a], Schedule a)
exec2 xs schedule = (xs, exec1 $ exec1 schedule)

instance Sortable ScheduledMergeSort where

    -- | Construct new (empty) collection in O(1).
    empty = SMS 0 []
    
    -- | Add new item to an existing sorted collection in O(log(n)) worst case time.
    add x (SMS size segs) = SMS (size + 1) (map (uncurry exec2) segs')
        where
            addSeg xs segs size rsched =
                if even size then (xs, reverse rsched) : segs
                else let ((xs', []):segs') = segs
                         xs'' = mrg xs xs'
                     in addSeg xs'' segs' (size `div` 2) (xs'':rsched)
            segs' = addSeg [x] segs size []
    
    -- | Convert a (sorted) collection into a sorted list in O(n) worst case steps.
    sort (SMS size segs) = mrgAll [] segs
        where
            mrgAll xs [] = xs
            mrgAll xs ((xs', _):segs) = mrgAll (mrg xs xs') segs

## Sets

|      instance      | persistence | empty |   member  |   insert  |
|:------------------:|:-----------:|:-----:|:---------:|:---------:|
| Binary Search Tree |  ephemeral  |  O(1) |    O(n)   |    O(n)   |
|   Red-Black Tree   |  ephemeral  |  O(1) | O(log(n)) | O(log(n)) |

In [ ]:
class Set s where

    -- | Construct new (empty) set
    empty :: Ord a => s a
    
    -- | Check whether a set contains given item
    member :: Ord a => a -> s a -> Bool
    
    -- | Add new item to a set while maintaining the item uniqueness property
    insert :: Ord a => a -> s a -> s a

### Unbalanced Set
Implementation of an unbalanced set via a *Binary Search Tree (BST)*.

In [ ]:
data Tree a = Empty | Node (Tree a) a (Tree a)

instance Set Tree where

    -- | Construct an empty set in O(1).
    empty = Empty
    
    -- | Check whether this set contains given item.
    -- | Since the underlying BST may not be balanced, this function may take O(n) steps in the worst case.
    member _ Empty = False
    member x (Node a y b) = case (compare x y) of
        EQ -> True
        LT -> member x a
        GT -> member x b
    
    -- | Add new item to this set if it's not present yet.
    -- | Similarly to 'member', for an unbalanced instance this may take up to O(n) steps.
    insert x Empty = Node Empty x Empty
    insert x s @ (Node a y b) = case (compare x y) of
        EQ -> s
        LT -> Node (insert x a) y b
        GT -> Node a y (insert x b)

### Balanced Set
Implementation of a balanced set via a [Red-Black Tree](https://en.wikipedia.org/wiki/Red%E2%80%93black_tree) without any fancy optimizations. Specifically, in `ins` (e.g. for the left child) dosn't have to check for all the red-red violations in `balance` (actually it does not have to check the color of any node not on the search path).

In [ ]:
data Color = R | B

data Tree a = Empty | Node Color (Tree a) a (Tree a)

-- | Re-balance and locally repair the RBT color property by pushing
-- | one of two consecutive red nodes with a black parent up the path to the root.
balance :: Color -> Tree a -> a -> Tree a -> Tree a
balance B (Node R (Node R a x b) y c) z d = Node R (Node B a x b) y (Node B c z d)
balance B (Node R a x (Node R b y c)) z d = Node R (Node B a x b) y (Node B c z d)
balance B a x (Node R (Node R b y c) z d) = Node R (Node B a x b) y (Node B c z d)
balance B a x (Node R b y (Node R c z d)) = Node R (Node B a x b) y (Node B c z d)
balance color a x b = Node color a x b

instance Set Tree where 

    -- | Construct an empty set in O(1).
    empty = Empty
    
    -- | Check whether this set contains given item.
    -- | Since the underlying RBT is balanced, this function takes O(log(n)) steps in the worst case.
    member _ Empty = False
    member x (Node _ a y b) = case (compare x y) of
        EQ -> True
        LT -> member x a
        GT -> member x b
    
    -- | Add new item to this set if it's not present yet.
    -- |
    -- | Call to 'insert' takes at most O(log(n)) steps because the tree is kept balanced by
    -- | 'balance' when backing up after adding new node and the fact that in a RB tree the deepest
    -- | leaf is at most twice as far from the root as the shallowest leaf is.
    insert x Empty = Node R Empty x Empty
    insert x s = let (Node _ a y b) = ins s in Node B a y b
        where
            ins Empty = Node R Empty x Empty
            ins s @ (Node color a y b) = case (compare x y) of
                EQ -> s
                LT -> balance color (ins a) y b
                GT -> balance color a y (ins b)